In [1]:
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import datetime


In [2]:
daily = "https://www.tn.gov/health/cedep/ncov.html"
viral = pd.read_html(daily)
df=viral[3]
testing = df.drop([10])
age_groups = testing.rename(columns={'Age Ranges of Confirmed Cases.1': 'Number_Cases'})
age_groups


,Age Ranges of Confirmed Cases,Number_Cases
0,0-10,24
1,11-20,107
2,21-30,566
3,31-40,346
4,41-50,351
5,51-60,368
6,61-70,254
7,71-80,144
8,80+,63
9,Pending,16


In [9]:
df2 = viral[5]
TN_county = df2[:-3]
TN_county=TN_county.rename(columns={'Patient county name': 'County'})
TN_county


,County,Positive,Negative
0,Anderson,10,129
1,Bedford,2,74
2,Benton,4,32
3,Bledsoe,2,8
4,Blount,25,124
...,...,...,...
90,Wayne,1,17
91,Weakley,1,55
92,White,1,59
93,Williamson,131,1097


In [10]:
TNpop = pd.read_csv("Population Estimates by County.csv")
TNpop= TNpop.drop([96])
TNpop = TNpop.drop([0])
TNpop["Population Estimates by County"]= TNpop["Population Estimates by County"].str.split(" County", n = 1, expand = True)
TNpop = TNpop.rename(columns={'Population Estimates by County': 'County'})
TN_county_populations = TNpop.rename(columns={'Unnamed: 1': 'Population'})
TN_county_populations["Population"] = TN_county_populations["Population"].str.replace(",","").astype(int)
TN_county_populations


,County,Population
1,Anderson,76978
2,Bedford,49713
3,Benton,16160
4,Bledsoe,15064
5,Blount,133088
...,...,...
91,Wayne,16673
92,Weakley,33328
93,White,27345
94,Williamson,238412


In [ ]:

data=viral[4]
data

In [11]:
TN_data = TN_county_populations.merge(TN_county,left_on='County',right_on='County', how='left')
TN_data


,County,Population,Positive,Negative
0,Anderson,76978,10.0,129.0
1,Bedford,49713,2.0,74.0
2,Benton,16160,4.0,32.0
3,Bledsoe,15064,2.0,8.0
4,Blount,133088,25.0,124.0
...,...,...,...,...
90,Wayne,16673,1.0,17.0
91,Weakley,33328,1.0,55.0
92,White,27345,1.0,59.0
93,Williamson,238412,131.0,1097.0


In [29]:
TN_data["Percentage of County Population"] = round((TN_data["Positive"]/TN_data["Population"])*100,3)
TN_data.reset_index(drop=True)
TN_data["Percent_Tested"] = round(((TN_data["Positive"]+TN_data["Negative"])/TN_data["Population"])*100,1)
TN_data=TN_data.sort_values("Percent_Tested", ascending=False)
TN_data=TN_data.dropna()
TN_data


,County,Population,Positive,Negative,Percentage of County Population,Percent_Tested
41,Houston,8201,1.0,70.0,0.012,0.9
18,Davidson,694144,391.0,3885.0,0.056,0.6
93,Williamson,238412,131.0,1097.0,0.055,0.5
82,Sumner,191283,184.0,692.0,0.096,0.5
94,Wilson,144657,39.0,477.0,0.027,0.4
...,...,...,...,...,...,...
44,Jefferson,54495,6.0,69.0,0.011,0.1
51,Lincoln,34366,1.0,45.0,0.003,0.1
50,Lewis,12268,2.0,16.0,0.016,0.1
47,Lake,7016,0.0,10.0,0.000,0.1


In [28]:
url2="https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv"
viral_times=pd.read_csv(url2)
TN=viral_times[viral_times["state"] == 'Tennessee']
TN=TN[TN["date"] == '2020-03-30']
TN=TN.rename(columns={'county':'County'})
TN

,date,County,state,fips,cases,deaths
21354,2020-03-30,Anderson,Tennessee,47001.0,9,0
21355,2020-03-30,Bedford,Tennessee,47003.0,1,0
21356,2020-03-30,Benton,Tennessee,47005.0,3,0
21357,2020-03-30,Bledsoe,Tennessee,47007.0,2,0
21358,2020-03-30,Blount,Tennessee,47009.0,10,0
...,...,...,...,...,...,...
21427,2020-03-30,Wayne,Tennessee,47181.0,1,0
21428,2020-03-30,Weakley,Tennessee,47183.0,1,0
21429,2020-03-30,White,Tennessee,47185.0,1,0
21430,2020-03-30,Williamson,Tennessee,47187.0,111,0


In [42]:
TN_deaths = TN_data.merge(TN,left_on='County',right_on='County', how='left')
TN_deaths.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 92 entries, 0 to 91
Data columns (total 11 columns):
County                             92 non-null object
Population                         92 non-null int32
Positive                           92 non-null float64
Negative                           92 non-null float64
Percentage of County Population    92 non-null float64
Percent_Tested                     92 non-null float64
date                               74 non-null object
state                              74 non-null object
fips                               74 non-null float64
cases                              74 non-null float64
deaths                             74 non-null float64
dtypes: float64(7), int32(1), object(3)
memory usage: 8.3+ KB


df_counties=viral[4]
file = datetime.datetime.today()
date = file[0:10]
date


In [40]:
TN_deaths=TN_deaths.drop(TN_deaths.columns[['state, fips']], inplace=True)
TN_deaths

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
# from pathlib import Path
# p = Path('C:/Users/clayf/Documents/HOC/Covid_predictions/TN/Resources/')
df_counties.to_csv('C:/Users/clayf/Documents/HOC/Covid_predictions/TN/Resources/' + date +'counties2.csv',index=False)

In [ ]:
US = "https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/cases-in-us.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov%2Fcoronavirus%2F2019-ncov%2Fcases-in-us.html"
us_data = pd.read_html(US)
us_total_cases = us_data[0]
us_total_cases.rename(columns={list(us_total_cases)[1]:'Total'}, inplace=True)
us_total_cases['Total']=us_total_cases['Total'].astype(int)
US = us_total_cases.iloc[3,1]

In [ ]:
date = datetime.datetime.today()
date_modify = str(date)
date_to_add = date_modify[0:10]
x['date'] = date_to_add



In [ ]:
TN_ratio = (TN/6700000)*100
US_ratio = (US/372000000)*100

print(TN_ratio)
print(US_ratio)